In [1]:
import os
import numpy as np
import tensorflow as tf
from keras import models, layers, regularizers

from loop import TrainingLoop
from batch_selection import windowed_batch_selector, sorting_batch_selector

In [2]:
# Loading the dataset from the files saved in the preprocessing notebook.
path = 'data/music_genres'
prefix = 'music_genres_'
X_train = np.load(os.path.join(path, prefix+'train_vectors.npy'))
y_train = np.load(os.path.join(path, prefix+'train_labels.npy'))
X_test  = np.load(os.path.join(path, prefix+'test_vectors.npy'))
y_test  = np.load(os.path.join(path, prefix+'test_labels.npy'))

In [3]:
def build_model():
    # Setting up the model.
    model = models.Sequential()
    model.add(layers.Dense(250, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))
    return model

In [4]:
def train(model, X_train, y_train, batch_selector, epochs):
    selector_str = {windowed_batch_selector: 'windowed', sorting_batch_selector: 'sorting', None: 'original'}
    print('\n\n'+selector_str[batch_selector]+'\n')
    # Put the model in our custom training loop.
    TrainingLoop(
        model,
        X_train,
        y_train,
        validation_split = 0.1,
        batch_size = 12,
        optimizer = tf.keras.optimizers.Adam(),
        loss_function = tf.keras.losses.CategoricalCrossentropy(),
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        batch_selection = batch_selector,
        log_file = os.path.join('logs',selector_str[batch_selector],'music_genres.csv')
    ).train(epochs)  # Training model.

In [5]:
def evaluate(model, X_test, y_test):
    # We still have to compile the model for the test evaluation.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # After compiling we can run the evaluation.
    model.evaluate(X_test, y_test)

In [6]:
# Train and evaluate models with different batch selection algorithms.
for batch_selector in [None, windowed_batch_selector, sorting_batch_selector]:
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model = build_model()
    train(model, X_train, y_train, batch_selector=batch_selector, epochs=50)
    evaluate(model, X_test, y_test)

	  0% | 0/60 [00:00<?, ?it/s]

original

Epoch 1/50	Loss: 1.8535	Metrics: 0.2055: 	Validation metrics: 0.375: 	100% | 60/60 [00:04<00:00, 13.91it/s]
Epoch 2/50	Loss: 1.5358	Metrics: 0.3638: 	Validation metrics: 0.4444: 	100% | 60/60 [00:00<00:00, 61.16it/s]
Epoch 3/50	Loss: 1.4155	Metrics: 0.4583: 	Validation metrics: 0.5138: 	100% | 60/60 [00:00<00:00, 63.90it/s]
Epoch 4/50	Loss: 1.6259	Metrics: 0.4819: 	Validation metrics: 0.4722: 	100% | 60/60 [00:00<00:00, 69.05it/s]
Epoch 5/50	Loss: 1.3714	Metrics: 0.5430: 	Validation metrics: 0.5138: 	100% | 60/60 [00:00<00:00, 65.79it/s]
Epoch 6/50	Loss: 1.4894	Metrics: 0.5722: 	Validation metrics: 0.5277: 	100% | 60/60 [00:00<00:00, 65.86it/s]
Epoch 7/50	Loss: 1.5168	Metrics: 0.5638: 	Validation metrics: 0.5277: 	100% | 60/60 [00:01<00:00, 54.95it/s]
Epoch 8/50	Loss: 1.6092	Metrics: 0.6013: 	Validation metrics: 0.5416: 	100% | 60/60 [00:01<00:00, 48.86it/s]
Epoch 9/50	Loss: 1.4755	Metrics: 0.6041: 	Validation metrics: 0.5833: 	100% | 60/60 [00: